# Import Packages

In [ ]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from functionalities import dataloader as dl
from functionalities import tracker as tk
from architecture import INN as inn
from functionalities import CIFAR_coder_loss as cl
from functionalities import trainer as tr
from functionalities import filemanager as fm
from functionalities import plot as pl
from functionalities import gpu 

# Pretraining Setup

In [ ]:
num_epoch = 8
batch_size = 32
latent_dim_lst = [2 ** x for x in range(0, 11, 2)]
#latent_dim = 300
number_dev = 0
lr_init = 1e-3
l2_reg  = 1e-6
milestones = [6, 7, 8]
modelname = 'celeba_INN_com_bottleneck'
get_model = inn.celeba_inn_com
image_size = 128

device = gpu.get_device(number_dev)

In [ ]:

IMAGE_PATH = './img_align_celeba/'
transform = transforms.Compose([
    #transforms.Scale(image_size),
    transforms.Resize(image_size),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
dataset = ImageFolder(IMAGE_PATH, transform)
trainloader, testloader = dl.split_dataset(dataset, 0.2, batch_size, False)

In [ ]:
model, model_params, track, MMD_loss = tr.init_model(get_model, latent_dim, 'l1', device, a_distr=0, a_disen=0)
optimizer, scheduler = tr.init_training(model_params, lr_init, l2_reg, milestones)

# Training

In [ ]:
model = tr.train_bottleneck(num_epoch, get_model, 'l1', modelname, milestones, latent_dim_lst, trainloader, None, 
                            testloader, a_distr=0, a_disen=0, lr_init=lr_init, l2_reg=l2_reg, device=device, 
                            save_model=True, num_img=25, grid_row_size=5)

# Plot Reconstruction and Difference Images Examples

In [ ]:
for lat_dim in latent_dim_lst:
    print("Latent Dimension: ", lat_dim)
    #print(modelname + "_{}".format(lat_dim), '{}_{}_{}'.format(modelname, lat_dim, num_epoch))
    #model = fm.load_model('{}_{}_{}'.format(modelname, lat_dim, num_epoch), modelname + "_{}".format(lat_dim))
    model = get_model().to(device)
    model = fm.load_weight(model, '{}_{}_{}'.format(modelname, lat_dim, num_epoch), '{}_bottleneck'.format(modelname))
    pl.plot_diff(model, trainloader, lat_dim, device, 25, 5)

# Plot Recontruction Loss against Bottleneck Size

In [ ]:
_, l1_rec_test, _, _, _ = fm.load_variable('bottleneck_test_loss_{}'.format(modelname), modelname)
_, l1_rec_train, _, _, _ = fm.load_variable('bottleneck_train_loss_{}'.format(modelname), modelname)

pl.plot(latent_dim_lst, [l1_rec_train, l1_rec_test], 'latent dimension', 'loss', ['train', 'test'], 'Test Reconstruction Loss History', '{}_bottleneck_History'.format(modelname)) 